# 03 - Sun Geometry Validator (OrbitCheck)

This notebook computes the expected sun position (azimuth + elevation) from:
- timestamp (UTC)
- location (lat/lon)

It then compares (optional) against any sun angles provided in metadata.

Outputs:
- Sun report JSON saved to: `data/outputs/reports/`


In [ ]:
!pip -q install astral

import os, json
from datetime import datetime, timezone
from astral import LocationInfo
from astral.sun import azimuth, elevation


In [ ]:
PROJECT_ROOT = "/content/orbitcheck"

META_DIR = os.path.join(PROJECT_ROOT, "data/raw/metadata")
REPORT_DIR = os.path.join(PROJECT_ROOT, "data/outputs/reports")

os.makedirs(REPORT_DIR, exist_ok=True)

print("META_DIR:", META_DIR)
print("REPORT_DIR:", REPORT_DIR)


In [ ]:
meta_files = [f for f in os.listdir(META_DIR) if f.endswith(".json")]
if len(meta_files) == 0:
    raise ValueError("No metadata JSON found. Run Notebook 01 first.")

print("Available metadata files:")
for f in meta_files:
    print(" -", f)

meta_filename = input("\nEnter metadata filename (example: sample1.json): ").strip()
meta_path = os.path.join(META_DIR, meta_filename)

with open(meta_path, "r") as f:
    meta = json.load(f)

print("\nLoaded metadata:")
print(json.dumps(meta, indent=2))


In [ ]:
def compute_sun_position(lat, lon, utc_time_str):
    dt = datetime.strptime(utc_time_str, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc)
    loc = LocationInfo(latitude=lat, longitude=lon)
    sun_az = azimuth(loc.observer, dt)
    sun_el = elevation(loc.observer, dt)
    return float(sun_az), float(sun_el)

lat = meta["location"]["lat"]
lon = meta["location"]["lon"]
timestamp = meta["timestamp_utc"]

expected_az, expected_el = compute_sun_position(lat, lon, timestamp)

print("\nExpected Sun Position from Physics")
print(f"Sun Azimuth:   {expected_az:.2f}°")
print(f"Sun Elevation: {expected_el:.2f}°")


In [ ]:
meta_az = meta.get("sun_metadata", {}).get("azimuth", None)
meta_el = meta.get("sun_metadata", {}).get("elevation", None)

def angular_difference(a, b):
    diff = abs(a - b) % 360
    return min(diff, 360 - diff)

comparison = {}

if meta_az is not None:
    comparison["azimuth_error_deg"] = angular_difference(expected_az, float(meta_az))
else:
    comparison["azimuth_error_deg"] = None

if meta_el is not None:
    comparison["elevation_error_deg"] = abs(expected_el - float(meta_el))
else:
    comparison["elevation_error_deg"] = None

print("\n Comparison with metadata sun angles:")
print(json.dumps(comparison, indent=2))


In [ ]:
sun_report = {
    "image_id": meta["image_id"],
    "timestamp_utc": timestamp,
    "location": {
        "lat": lat,
        "lon": lon
    },
    "expected_sun": {
        "azimuth_deg": expected_az,
        "elevation_deg": expected_el
    },
    "metadata_sun": {
        "azimuth_deg": meta_az,
        "elevation_deg": meta_el
    },
    "errors": comparison
}

out_path = os.path.join(REPORT_DIR, f"{meta['image_id']}_sun_report.json")
with open(out_path, "w") as f:
    json.dump(sun_report, f, indent=2)

print("\n Sun report saved:", out_path)
print(json.dumps(sun_report, indent=2))
